In [ ]:
! pip install pytorch-lightning

In [8]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm
import pdb
import pytorch_lightning as pl

In [16]:
class ResNet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28 * 28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)
    self.loss = nn.CrossEntropyLoss()
  
  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2 +h1)
    logits = self.l3(do)
    return logits

  def configure_optimizers(self):
    optimizer = optim.SGD(self.parameters, lr=1e-2)

  def training_step(self, batch, batch_idx):
    x, y = batch
    # x = batchsize * 1 * 28 * 28
    b = x.size(0)
    x = x.view(b, -1)
    # el -1 multiplica los otros numeros no declarados

    # 1) forward
    # l = logit = funcion que devuelve una probabilidad entre 0 y 1 (log odds function) el output de la ultima capa
    logits = self(x)
    pdb.set_trace()
    # 2) compute the objective function
    # the loss is the distance from the network prediction and the real value
    # the objective funtion is equal to the loss in value but it's a function of it's parameters not the output
    J = self.loss(logits, y)

    return {'loss': J}

  def training_dataloader():
    # esto acopla la data al modelo lo cual no siempre es deseable 
    # ponele que estas entrenando un imageNet para clasificar 5 clases
    # esta bueno que eso este definido en el modelo, queda lindo y limpio
    training_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
    # train, val = random_split(training_data, [55000, 5000])
    train_loader = DataLoader(training_data, batch_size=32)
    # val_loader = DataLoader(val, batch_size=32)
    return train_loader

model = ResNet()

In [17]:
class ImageClassifier(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.resnet = ResNet()
